# Data Merging and Cleaning

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from functools import reduce

In [2]:
hh = pd.read_csv('data/hhpub.csv')
per = pd.read_csv('data/perpub.csv')
veh = pd.read_csv('data/vehpub.csv')
trip = pd.read_csv('data/trippub.csv')

The original NHTS data consists of four components: household, person, vehicle, and trip. Our research question, "how does retirement change elderly people's driving habits," led us to focus on four specific household types:
1. Households with two adults and no children (LC2)
2. Households with two retired adults and no children (LC10)
3. Households with one adult and no children (LC1)
4. Households with one retired adult and no children (LC9)

In [3]:
# Take out all candidate household IDs that fall into any of these four categories
candid_HOUSEID = hh[(hh['LIF_CYC'] == 2) | (hh['LIF_CYC'] == 10)| 
                    (hh['LIF_CYC'] == 1) | (hh['LIF_CYC'] == 9)].loc[:, "HOUSEID"]

# Filter out the person file that falls into the candidate group
candid_PER = per[per['HOUSEID'].isin(candid_HOUSEID)]

# Group by 'HOUSEID' and obtain the minimum age for each 'HOUSEID'
candid_minAGE = candid_PER.groupby('HOUSEID')['R_AGE'].min()

# Filter out HOUSEIDs where the minimum age falls into 55 and 70
target_HOUSEID = candid_minAGE[(55 <= candid_minAGE) & (candid_minAGE <= 70)].index

# Merging

We predetermined what to do with all available variables: https://docs.google.com/spreadsheets/d/1pwVraaNGrRI4rigtJ6Vo7o9EJiQHxv_TXCSRkMWVnt0/edit#gid=1070989035

In [4]:
def summarize_pH(lower, target_var, agg_funcs, group_var='HOUSEID'):
    """
    Summarizes lower-level data by household and returns aggregated features.

    Parameters:
        lower (DataFrame): Lower-level DataFrame.
        group_var (str): Name of the column containing household IDs.
        target_var (str): Name of the column containing the target variable to summarize.
        agg_funcs (list or str): List of aggregation functions to apply or a single aggregation function.
                                 For instance [mean].

    Returns:
        DataFrame: DataFrame containing aggregated features by household.
    """
    if isinstance(agg_funcs, str):
        agg_funcs = [agg_funcs]

    # Group by household ID and compute summary statistics for the target variable
    summary_df = lower.groupby(group_var)[target_var].agg(agg_funcs).reset_index()

    # Rename columns to include aggregation function suffix
    new_cols = [group_var] + [f"{target_var}_{agg.__name__}" for agg in agg_funcs]
    summary_df.columns = new_cols

    return summary_df

## Household Profile

In [5]:
# Filter hh DataFrame to include only the target HOUSEIDs
hh_target = hh[hh['HOUSEID'].isin(target_HOUSEID)]

In [6]:
# Select out the variables that we previously determined to be informative for this analysis
codebook = pd.read_csv('https://docs.google.com/spreadsheets/d/1pwVraaNGrRI4rigtJ6Vo7o9EJiQHxv_TXCSRkMWVnt0' + 
                       '/export?format=csv&gid=653078334').dropna()

hh_final = hh_target[codebook['NAME']]

In [7]:
hh_final

,HHFAMINC_IMP,HOMETYPE,URBRUR_2010,CDIVMSAR,CENSUS_R,DRVRCNT,HHSIZE,HHVEHCNT,HH_HISP,HH_RACE,HOMEOWN,HOUSEID,LIF_CYC,MSACAT,MSASIZE,RAIL,URBAN,URBANSIZE,URBRUR,WRKCOUNT
9,10,1,1,11,1,2,2,2,2,1,2,9000013046,10,1,4,1,1,4,1,0
12,6,1,2,13,1,2,2,3,2,1,2,9000013081,10,3,3,2,4,6,2,1
13,7,1,1,11,1,2,2,2,2,3,2,9000013083,10,1,5,1,4,6,2,0
18,6,1,1,11,1,1,1,1,2,1,2,9000013145,1,1,4,1,1,4,1,1
19,6,1,2,13,1,2,2,2,2,1,1,9000013148,10,3,2,2,4,6,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7875,8,1,2,93,4,1,1,3,1,1,1,9000217837,1,3,1,2,4,6,2,1
7878,5,1,2,64,3,2,2,4,2,1,4,9000217865,10,4,6,2,4,6,2,0
7884,6,1,1,62,3,2,2,2,2,1,1,9000217969,2,2,4,2,1,3,1,2
7890,4,1,2,64,3,2,2,2,2,1,2,9000218034,10,4,6,2,4,6,2,0


## Vehicle Profile

In [8]:
# Filter veh DataFrame to include only the target HOUSEIDs
veh_target = veh[veh['HOUSEID'].isin(target_HOUSEID)]

# Select out the variables that we previously determined to be informative for this analysis
codebook = pd.read_csv('https://docs.google.com/spreadsheets/d/1pwVraaNGrRI4rigtJ6Vo7o9EJiQHxv_TXCSRkMWVnt0' + 
                       '/export?format=csv&gid=1079478092').dropna()

In [9]:
codebook.AGGREGATE.value_counts()

AGGREGATE
mean() & > 0    2
mean            1
Name: count, dtype: int64

In [10]:
# Write a function to aggregate the mean non-negative value
def mean_nonnegative(x):
    """
    Calculate the mean of non-negative values in a Series.

    Parameters:
        x (pandas.Series): Input Series.

    Returns:
        float: Mean of non-negative values, or 0 if there are only negative values.
    """
    non_negative_values = x[x >= 0]
    if non_negative_values.empty:
        return 0
    else:
        return non_negative_values.mean()

In [11]:
# Aggregate all necessary columns correspondingly
tomean = codebook[codebook['AGGREGATE'] == 'mean'].loc[:, 'NAME'].tolist()
tonnmean = codebook[codebook['AGGREGATE'] == 'mean() & > 0'].loc[:, 'NAME'].tolist()

temp1 = [summarize_pH(veh_target, var, agg_funcs=[mean_nonnegative]) for var in tonnmean]
temp2 = [summarize_pH(veh_target, var, agg_funcs=[np.mean]) for var in tomean]

veh_final = reduce(lambda x, y: x.merge(y, on='HOUSEID'), temp1 + temp2)

By the way, there are only 2239 family that owns at least one car so the shape of this dataframe is different from the hh_final.

In [12]:
veh_final

,HOUSEID,ANNMILES_mean_nonnegative,COMMERCIALFREQ_mean_nonnegative,VEHAGE_mean
0,9000013046,7250.000000,0.0,4.000000
1,9000013081,6666.666667,0.0,6.333333
2,9000013083,12000.000000,0.0,4.500000
3,9000013145,5000.000000,0.0,6.000000
4,9000013148,9000.000000,0.0,2.000000
...,...,...,...,...
2234,9000217837,2400.000000,0.0,25.333333
2235,9000217865,2250.000000,0.0,18.500000
2236,9000217969,13500.000000,0.0,11.500000
2237,9000218034,5000.000000,0.0,11.500000


## Person Profile

In [13]:
# Filter veh DataFrame to include only the target HOUSEIDs
per_target = per[per['HOUSEID'].isin(target_HOUSEID)]

# Select out the variables that we previously determined to be informative for this analysis
codebook = pd.read_csv('https://docs.google.com/spreadsheets/d/1pwVraaNGrRI4rigtJ6Vo7o9EJiQHxv_TXCSRkMWVnt0' + 
                       '/export?format=csv&gid=1952500799').drop('Note', axis=1).dropna()

In [14]:
codebook.AGGREGATE.value_counts()

AGGREGATE
min() & > 0     54
mean() & > 0    11
cross-factor     6
sum() & > 0      5
min              2
min, max         1
Name: count, dtype: int64

In [15]:
# Write a function to aggregate the minimum non-negative value
def min_nonnegative(x):
    """
    Calculate the mean of non-negative values in a Series.

    Parameters:
        x (pandas.Series): Input Series.

    Returns:
        float: Mean of non-negative values, or 0 if there are only negative values.
    """
    non_negative_values = x[x >= 0]
    if non_negative_values.empty:
        return 0
    else:
        return non_negative_values.min()
    
# Write a function to aggregate the sum non-negative value
def sum_nonnegative(x):
    """
    Calculate the mean of non-negative values in a Series.

    Parameters:
        x (pandas.Series): Input Series.

    Returns:
        float: Mean of non-negative values, or 0 if there are only negative values.
    """
    non_negative_values = x[x >= 0]
    if non_negative_values.empty:
        return 0
    else:
        return non_negative_values.sum()

In [16]:
# Aggregate all necessary columns correspondingly except the ones for cross-factor
tomin = codebook[(codebook['AGGREGATE'] == 'min') | (codebook['AGGREGATE'] == 'min, max')].loc[:, 'NAME'].tolist()
tomax = codebook[codebook['AGGREGATE'] == 'min, max'].loc[:, 'NAME'].tolist()
tonnsum = codebook[codebook['AGGREGATE'] == 'sum() & > 0'].loc[:, 'NAME'].tolist()
tonnmean = codebook[codebook['AGGREGATE'] == 'mean() & > 0'].loc[:, 'NAME'].tolist()
tonnmin = codebook[codebook['AGGREGATE'] == 'min() & > 0'].loc[:, 'NAME'].tolist()

temp1 = [summarize_pH(per_target, var, agg_funcs=[np.min]) for var in tomin]
temp2 = [summarize_pH(per_target, var, agg_funcs=[np.max]) for var in tomax]
temp3 = [summarize_pH(per_target, var, agg_funcs=[sum_nonnegative]) for var in tonnsum]
temp4 = [summarize_pH(per_target, var, agg_funcs=[mean_nonnegative]) for var in tonnmean]
temp5 = [summarize_pH(per_target, var, agg_funcs=[min_nonnegative]) for var in tonnmin]

In [17]:
# Aggregate the cross-factor variables manually
tocross = codebook[codebook['AGGREGATE'] == 'cross-factor'].loc[:, 'NAME'].tolist()

# Write a function to summarize the factor proportion
def factor_proportions(df, factor_var, group_var='HOUSEID'):
    """
    Calculate the proportions of each factor level within each group.

    Parameters:
        df (pandas.DataFrame): Input DataFrame.
        group_var (str): Name of the column containing the group variable.
        factor_var (str): Name of the column containing the factor variable.

    Returns:
        pandas.DataFrame: DataFrame containing the proportions of each factor level within each group.
    """
    # Group by the group variable and calculate frequency of each factor level
    group_counts = df.groupby(group_var)[factor_var].value_counts().unstack(fill_value=0)
    
    # Normalize the counts by row count of each group
    group_proportions = group_counts.div(group_counts.sum(axis=1), axis=0)
    
    # Rename the columns with factor level proportions
    group_proportions.columns = [f'{factor_var}_{level}_PROP' for level in group_proportions.columns]
    
    return group_proportions.reset_index()

# Generate the summary file for cross-factor variables
temp6 = [factor_proportions(per_target, var) for var in tocross]

In [18]:
per_final = reduce(lambda x, y: x.merge(y, on='HOUSEID'), temp1 + temp2 + temp3 + temp4 + temp5 + temp6)

In [19]:
per_final

,HOUSEID,EDUC_amin,FRSTHM_amin,OUTOFTWN_amin,EDUC_amax,DELIVER_sum_nonnegative,DELIV_FOOD_sum_nonnegative,DELIV_GOOD_sum_nonnegative,DELIV_GROC_sum_nonnegative,DELIV_PERS_sum_nonnegative,...,R_RACE_IMP_97_PROP,R_SEX_IMP_1_PROP,R_SEX_IMP_2_PROP,WORKER_1_PROP,WORKER_2_PROP,WRKLOC_-1_PROP,WRKLOC_1_PROP,WRKLOC_2_PROP,WRKLOC_3_PROP,WRKLOC_4_PROP
0,9000013046,6,1,2,6,15,0,16,0,4,...,0.0,0.5,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,9000013081,3,1,2,3,12,0,14,0,0,...,0.0,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0
2,9000013083,7,1,2,7,6,0,6,0,0,...,0.0,0.5,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,9000013145,5,1,2,5,1,0,0,0,0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,9000013148,5,1,2,6,2,0,1,0,0,...,0.0,0.5,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2358,9000217837,6,1,2,6,9,0,9,0,0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2359,9000217865,4,1,2,4,10,0,10,0,0,...,0.0,0.5,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2360,9000217969,3,1,2,4,10,0,10,0,0,...,0.0,0.5,0.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2361,9000218034,1,1,2,2,20,0,20,0,0,...,0.0,0.5,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.0


## Final Merging

In [20]:
final = reduce(lambda x, y: x.merge(y, on='HOUSEID', how='left'), [hh_final, veh_final, per_final]).fillna(0)

Here are the final tasks:
- Create two dummy variables: RETIRE and SINGLE. RETIRE indicates whether the individuals in the household have retired, while SINGLE indicates whether the household consists of only one person.
- Drop the variables LIF_CYC and HOUSEID. LIF_CYC would be collinear with the new dummy variables, and HOUSEID should not be included in the analysis.

In [21]:
# Create two dummies
final['RETIRE'] = final['LIF_CYC'].isin([9, 10])
final['SINGLE'] = final['LIF_CYC'].isin([1, 9])

# Drop LIF_CYC and HOUSEID
final = final.drop(['LIF_CYC', 'HOUSEID'], axis=1)

In [22]:
# Write the final.csv for estimation
final.to_csv('data/final.csv', index=False)  

In [23]:
final

,HHFAMINC_IMP,HOMETYPE,URBRUR_2010,CDIVMSAR,CENSUS_R,DRVRCNT,HHSIZE,HHVEHCNT,HH_HISP,HH_RACE,...,R_SEX_IMP_2_PROP,WORKER_1_PROP,WORKER_2_PROP,WRKLOC_-1_PROP,WRKLOC_1_PROP,WRKLOC_2_PROP,WRKLOC_3_PROP,WRKLOC_4_PROP,RETIRE,SINGLE
0,10,1,1,11,1,2,2,2,2,1,...,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.0,True,False
1,6,1,2,13,1,2,2,3,2,1,...,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,True,False
2,7,1,1,11,1,2,2,2,2,3,...,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.0,True,False
3,6,1,1,11,1,1,1,1,2,1,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,False,True
4,6,1,2,13,1,2,2,2,2,1,...,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2358,8,1,2,93,4,1,1,3,1,1,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,False,True
2359,5,1,2,64,3,2,2,4,2,1,...,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.0,True,False
2360,6,1,1,62,3,2,2,2,2,1,...,0.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,False,False
2361,4,1,2,64,3,2,2,2,2,1,...,0.5,0.0,1.0,1.0,0.0,0.0,0.0,0.0,True,False
